# **CLOTHING SIMILARITY**

# **SEMANTIC SEARCH**

In [2]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=8ecd28a7c4956ba234239745792e7f8a28953e2fac01bb9b77dfddefdacf298a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [3]:
!pip3 install pinecone-client


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 30.8 MB/s eta 0:00:00


In [19]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import string
import spacy

import pinecone
import pandas as pd
import nltk 
import re
import nltk

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.tokenize import word_tokenize

en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words


from sentence_transformers import SentenceTransformer



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Sentence Transformer model

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2') #384

# LOADING DATASET

In [14]:
df_Ajio = pd.read_csv('/content/drive/MyDrive/mercor/Ajio_cloth.csv')
df_Myntra = pd.read_csv('/content/drive/MyDrive/mercor/Myntra_clothing.csv')

# CLEANING OF DATASET

In [ ]:
# Clean Text
def clean_text(text):
#replace the html characters with " "
    text=re.sub('<.*?>', ' ', text)  
    
#remove the punctuations
    text = text.translate(str.maketrans(' ',' ',string.punctuation))
    
#consider only alphabets and numerics
    text = re.sub('[^a-zA-Z0-9]',' ',text)  
    
#replace newline with space
    text = re.sub("\n"," ",text)
    
#convert to lower case
    text = text.lower()
    
#split and join the words
    text=' '.join(text.split())
    return text

def lemmatize_words(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word,pos='v') for word in words]
    return ' '.join(words)

In [ ]:
def clean(data,word):
    #Cleaning
    data[word] = data[word].apply(lambda x: ''.join([w for w in clean_text(x)]))
    #Removing stopwords 
    data[word] = data[word].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
    #Lemmatization
    data[word] = data[word].apply(lemmatize_words)

In [ ]:
clean(df_Ajio ,'Description')

In [ ]:
df_Ajio


,Unnamed: 0,URL,Description
0,0,https://www.ajio.com/netplay-checked-polo-t-sh...,check polo tshirt
1,1,https://www.ajio.com/netplay-tapered-fit-flat-...,taper fit flatfront trousers
2,2,https://www.ajio.com/the-indian-garage-co-stri...,strip slim fit shirt patch pocket
3,3,https://www.ajio.com/performax-heathered-crew-...,heathered crewneck tshirt
4,4,https://www.ajio.com/john-players-jeans-washed...,wash skinny fit jeans whisker
...,...,...,...
367167,367167,https://www.ajio.com/indie-picks-woven-pochamp...,weave pochampally ikat dress material
367168,367168,https://www.ajio.com/indie-picks-woven-pochamp...,weave pochampally ikat dress material
367169,367169,https://www.ajio.com/griva-designer-floral-emb...,floral embroider saree
367170,367170,https://www.ajio.com/global-desi-pleated-dhoti...,pleat dhoti pant semielasticated waist


In [ ]:

clean(df_Myntra , 'Description')

In [15]:
df_Myntra


,Unnamed: 0,URL,Description
0,0,https://www.myntra.com/jeans/roadster/roadster...,roadster men navy blue slim fit mid rise clean...
1,1,https://www.myntra.com/track-pants/locomotive/...,locomotive men black white solid slim fit tra...
2,2,https://www.myntra.com/shirts/roadster/roadste...,roadster men navy white black geometric print...
3,3,https://www.myntra.com/shapewear/zivame/zivame...,zivame women black saree shapewear zi3023core0...
4,4,https://www.myntra.com/tshirts/roadster/roadst...,roadster women white solid v neck pure cotton ...
...,...,...,...
526559,526559,https://www.myntra.com/sweaters/hm/hm-women-be...,hm women beige oversized cashmere jumper
526560,526560,https://www.myntra.com/jackets/hm/hm-women-bla...,hm women black water repellent parka jacket
526561,526561,https://www.myntra.com/dresses/250-designs/250...,250 designs blue lace sheath dress
526562,526562,https://www.myntra.com/dresses/masaba/masaba-w...,masaba women yellow printed a line dress


# SENTENCE Transformation

EXAMPLE OF VECTOR LENGTH 384

In [18]:
model.encode("blue jeans and shirt")

array([-1.06921680e-01,  6.27289414e-02,  1.08112805e-02,  2.33791433e-02,
        3.24305594e-02,  1.88235193e-02,  1.21957123e-01, -3.14250588e-02,
       -2.00123657e-02,  6.74599558e-02,  4.67776656e-02, -3.34801599e-02,
        4.03832681e-02, -2.72109695e-02,  5.71504682e-02,  8.08655620e-02,
        8.86838064e-02, -1.04886461e-02, -7.12974742e-02, -4.74999547e-02,
       -3.84469181e-02,  5.05018942e-02,  8.99032131e-03,  4.72770818e-02,
       -9.22948048e-02,  2.17274968e-02, -2.69456152e-02,  7.37936189e-03,
       -2.79861055e-02, -3.87885459e-02, -2.41875313e-02,  2.35154163e-02,
        2.37383991e-02, -1.51648708e-02, -2.16656625e-02, -8.59949067e-02,
        7.80301690e-02, -3.25806774e-02, -3.41862850e-02,  1.82408746e-02,
       -4.40457761e-02, -8.26873407e-02, -1.48159042e-02,  2.90158391e-02,
        3.15234549e-02,  2.30227504e-02,  6.83920681e-02,  9.03505310e-02,
        5.78995533e-02,  3.30615677e-02,  3.18898484e-02,  2.53971200e-02,
       -4.40532900e-02,  

# Connection with Vector Database PineCone

In [21]:
pinecone.init(api_key='a50af583-55c1-4cfc-8798-db0b892882ca' ,environment = 'us-west4-gcp-free')
pinecone_index = pinecone.Index(index_name='clothdesc')



# Adding Data to Database

In [ ]:
#transforming Text to vectors and adding our data to database
#adding data to Pinecone vector Database
data = df_Ajio.iloc[0:100000]
embed = []
for i , row in data.iterrows():
  embed.append(
      (
          row['URL'], 
          model.encode(row['Description']).tolist()
      )
  )
  if len(embed) == 50 or len(embed) == len(data):
    pinecone_index.upsert(vectors = embed) 
    embed = [] 

In [ ]:
data1 = df_Myntra.iloc[0:100000]
embed = []
for i , row in data1.iterrows():
  embed.append(
      (
          row['URL'], 
          model.encode(row['Description']).tolist()
      )
  )
  if len(embed) == 50 or len(embed) == len(data):
    pinecone_index.upsert(vectors = embed) 
    embed = []

# Finding Output

In [22]:
query = "FLORAL DRESS pink"
xq = model.encode([query]).tolist()
result = pinecone_index.query(xq ,top_k = 10 , include_metadata=True)
result

{'matches': [{'id': 'https://www.myntra.com/dresses/winered/winered-pink-floral-a-line-dress/15952358/buy',
              'score': 0.843700349,
              'values': []},
             {'id': 'https://www.myntra.com/dresses/urbanic/urbanic-pink--white-ditsy-floral-wrap-dress/15629254/buy',
              'score': 0.82325381,
              'values': []},
             {'id': 'https://www.myntra.com/dresses/rustorange/rustorange-pink-floral-a-line-dress/16917306/buy',
              'score': 0.819204867,
              'values': []},
             {'id': 'https://www.myntra.com/dresses/all-about-you/all-about-you-black--pink-floral-printed-a-line-dress/14298310/buy',
              'score': 0.80373764,
              'values': []},
             {'id': 'https://www.myntra.com/dresses/all-about-you/all-about-you-women-pretty-pink-floral-tiered-dress/14381474/buy',
              'score': 0.803330123,
              'values': []},
             {'id': 'https://www.myntra.com/dresses/dressberry/dress

In [24]:
query = " yellow  dress"
xq = model.encode([query]).tolist()
result = pinecone_index.query(xq ,top_k = 10 , include_metadata=True)
result

{'matches': [{'id': 'https://www.myntra.com/dresses/dressberry/dressberry-yellow-shirt-dress/15424536/buy',
              'score': 0.841076612,
              'values': []},
             {'id': 'https://www.myntra.com/dresses/herenow/herenow-yellow--white-floral-printed-wrap-dress/13651964/buy',
              'score': 0.803939342,
              'values': []},
             {'id': 'https://www.myntra.com/dresses/all-about-you/all-about-you-blue--yellow-printed-a-line-dress/14381154/buy',
              'score': 0.803457558,
              'values': []},
             {'id': 'https://www.myntra.com/dresses/w/w-women-yellow-printed-a-line-dress/13646374/buy',
              'score': 0.755458236,
              'values': []},
             {'id': 'https://www.myntra.com/dresses/herenow/herenow-women-yellow-printed-a-line-dress/13748486/buy',
              'score': 0.751205325,
              'values': []},
             {'id': 'https://www.myntra.com/dresses/urbanic/urbanic-yellow--black-printed-sli